In [1]:
import sqlite3, json

In [2]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error:", e)
    finally:
        return conn

In [3]:
def create_table(conn, table, columns):
    try:
        sql = f'Drop table if exists {table};'
        c = conn.cursor()
        c.execute(sql)
        conn.commit()
    except Exception as e:
        print('Table drop error:',e)
        
    try:
        sql = f'Create table if not exists {table} ('
        for column in columns:
            sql += f'{column[0]} {column[1]},'
        sql = sql[:-1] # remove trailing comma
        sql += ');'
        c = conn.cursor()
        c.execute(sql)
        conn.commit()
        print('Table created:', table)
    except sqlite3.Error as e:
        print("Table creation error:", e)

In [19]:
def insert_records(conn, table, columns, values):
    try:
        c = conn.cursor()
        sql = f'INSERT INTO {table} VALUES '
        for valset in values:
            sql += "("
            for val in valset:
                if isinstance(val, str): 
                    val = val.replace("'","''") # escape any single quote entries
                sql+= f"'{val}',"
            sql = sql[:-1] # remove last comma
            sql += "),"
        sql = sql[:-1] + ';' # remove last comma
        c.execute(sql)
        conn.commit()
        print('Inserted',len(values),'values')
    except sqlite3.Error as e:
        print("Record Insertion error:", e)

In [189]:
with open('./assets/default_rolltables.json') as f:
    rolltables = json.load(f)

In [190]:
db = 'rolltables.db'
conn = create_connection(db)
for rolltable in rolltables:
    table = rolltable['table']
    columns = rolltable['columns']
    values = rolltable['values']
    create_table(conn, table, columns)
    insert_records(conn, table, columns, values)
conn.close()

Table created: PlayerRace
Inserted 20 values
Table created: PlayerPersonality
Inserted 20 values
Table created: PlayerClass
Inserted 12 values
Table created: Environment
Inserted 12 values
Table created: StartArea
Inserted 10 values
Table created: DungeonType
Inserted 10 values
Table created: Passages
Inserted 10 values
Table created: PassageWidth
Inserted 9 values
Table created: Door
Inserted 10 values
Table created: DoorBeyond
Inserted 5 values
Table created: Stair
Inserted 14 values


## Radom Select testing

In [63]:
def query(table, columns=['id','value']):
    sql = 'Select '
    for col in columns:
        sql += col + ", "
    sql = sql[:-2]
    sql += f' from {table} order by wieght * round(abs(random()/92233720368547758.07),3)/100 DESC LIMIT 1;'
    conn = sqlite3.connect('rolltables.db')
    c = conn.cursor()
    c.execute(sql)
    results = c.fetchone()
    conn.close()
    return results
    

In [188]:
r =''
i = 0
while r != 'Goliath' or i >10000:
    i+=1
    results = query('PlayerRace')
    r = results[1]
print(i)

8402


## List of tables

In [ ]:
single_value_table_names = [
    'PlayerRace'
    ,'PlayerPersonality' 
    ,'PassageWidth'
    ,'Door'
    ,'Stair'
    ,'Environment'
    ,'ChamberExits'
    ,'ChamberExitsLarge'
    ,'ChamberExitLocation'
    ,'ChamberState'
    ,'ChamberPurpose'
    ,'ChamberPurposeDeath'
    ,'ChamberPurposeLair'
    ,'ChamberPurposeMaze'
    ,'ChamberPurposeMine'
    ,'ChamberPurposeTreasure'
    ,'ChamberPurposeTomb'
    ,'ChamberPurposeStrong'
    ,'ChamberPurposeTemple'
    ,'ChamberPurposeGate'
    ,'ChamberPurpose'
    ,'ChamberHazard'
    ,'ChamberObstacle'
    ,'TrapSeverity'
    ,'TrapTrigger'
    ,'TrapEffect'
    ,'TrickObject'
    ,'TrickEffect'
    ,'MonsterMotivation'
    
]

In [1]:
Multi_value_tables = [
    {'PlayerClass': ['Value','PrimaryStat','SecondaryStat','Proficencies']},
    {'Background': ['Value','Skills','Languages','Tools','Source','Page']},
    {'StartArea': ['Value','Passages','Doors']},
    {'Passage': ['Value','Chamber','Stair','Door','Passage']},
    {'DoorBeyond': ['Value','Chamber','Stair']},
    {'ChamberSize': ['Value','Large']},
    {'ExitType': ['Value','Door']},
    {'ChamberContents': ['Value','Type','Level']},
    {'Monster': ['Name','Environment','Size','Type','Tags','Alignment','Challenge','XP','Source','Page']},
]

## Weighted simulation testing

In [15]:
import random

In [153]:
def one_round(dic):
    winner = ["None",0]
    for k, v in dic.items():
        val = random.random()*v
        if val > winner[1]:
            winner[0] = k
            winner[1] = val
    return winner[0]

def simulation(rounds=100):
    i=1
    roll_table = {
        "5ft":2,
        "10ft":10,
        "20ft":2,
        "30ft":2,
        "40a":1,
        "40b":1,
        "40c":1,
        "40d":1,
        }
    winner_list = dict.fromkeys(roll_table, 0)

    for i in range(rounds):
        winner_list[one_round(roll_table)]+=1
    print(winner_list)

In [154]:
for i in range(20):
    simulation()

{'5ft': 7, '10ft': 82, '20ft': 5, '30ft': 6, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 6, '10ft': 85, '20ft': 4, '30ft': 4, '40a': 0, '40b': 1, '40c': 0, '40d': 0}
{'5ft': 8, '10ft': 85, '20ft': 3, '30ft': 4, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 3, '10ft': 87, '20ft': 2, '30ft': 7, '40a': 0, '40b': 1, '40c': 0, '40d': 0}
{'5ft': 4, '10ft': 89, '20ft': 6, '30ft': 1, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 2, '10ft': 90, '20ft': 5, '30ft': 3, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 6, '10ft': 81, '20ft': 6, '30ft': 6, '40a': 0, '40b': 0, '40c': 1, '40d': 0}
{'5ft': 5, '10ft': 83, '20ft': 6, '30ft': 4, '40a': 1, '40b': 0, '40c': 0, '40d': 1}
{'5ft': 5, '10ft': 83, '20ft': 4, '30ft': 7, '40a': 1, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 3, '10ft': 85, '20ft': 6, '30ft': 6, '40a': 0, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 7, '10ft': 88, '20ft': 2, '30ft': 2, '40a': 1, '40b': 0, '40c': 0, '40d': 0}
{'5ft': 5, '10ft': 87, '20ft': 5, '30ft': 3, '40a': 0, '40b': 0, 